In [1]:
from sentence_transformers import SentenceTransformer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import xgboost as xgb
import pandas as pd
import numpy as np

c:\Users\user\anaconda3\envs\skn\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_excel(r'C:\Users\user\Desktop\SKN_AFTER_STUDY\data\retest_augmentation.xlsx')

# re_category2에서만 중립 데이터 제거 (re_category1은 중립 유지)
print(f"데이터 필터링 전: {len(data)}개")
print(f"re_category1에서 중립: {(data['re_category1'] == '중립').sum()}개")
print(f"re_category2에서 중립: {(data['re_category2'] == '중립').sum()}개")

# re_category2에서만 중립 데이터 제거 (Category2에 중립이 없으므로 라벨 불일치 방지)
original_count = len(data)
data = data[data['re_category2'] != '중립'].copy()

# 인덱스 재설정
data = data.reset_index(drop=True)

print(f"데이터 필터링 후: {len(data)}개")
print(f"제거된 데이터: {original_count - len(data)}개")

# 필터링된 데이터 확인
print(f"남은 re_category1 클래스 ({len(data['re_category1'].unique())}개): {sorted(data['re_category1'].unique())}")
print(f"남은 re_category2 클래스 ({len(data['re_category2'].unique())}개): {sorted(data['re_category2'].unique())}")

# 중립 확인
print(f"필터링 후 re_category1 중립: {(data['re_category1'] == '중립').sum()}개")
print(f"필터링 후 re_category2 중립: {(data['re_category2'] == '중립').sum()}개")

data.head()

데이터 필터링 전: 3360개
re_category1에서 중립: 3개
re_category2에서 중립: 1개
데이터 필터링 후: 3359개
제거된 데이터: 1개
남은 re_category1 클래스 (10개): ['기쁨', '두려움', '미움(상대방)', '분노', '사랑', '수치심', '슬픔', '싫어함(상태)', '욕망', '중립']
남은 re_category2 클래스 (64개): ['갈등', '감동', '걱정', '경멸', '고마움', '고통', '공감', '공포', '궁금함', '귀중함', '그리움', '기대감', '난처함', '날카로움', '냉담', '너그러움', '놀람', '다정함', '답답함', '동정(슬픔)', '두근거림', '만족감', '매력적', '무기력', '미안함', '반가움', '반감', '발열', '부끄러움', '불만', '불신감', '불쾌', '불편함', '비위상함', '사나움', '수치심', '시기심', '신뢰감', '신명남', '실망', '싫증', '심심함', '아쉬움', '아픔', '안정감', '억울함', '외로움', '외면', '욕심', '원망', '위축감', '자랑스러움', '자신감', '절망', '죄책감', '즐거움', '초조함', '치사함', '타오름', '통쾌함', '편안함', '허망', '호감', '후회']
필터링 후 re_category1 중립: 2개
필터링 후 re_category2 중립: 0개


,Unnamed: 0,generator_context,category1,category2,input_context,original_index,augmentation_index,re_category1,re_category2
0,0,갑자기 내 책상 위에 놓인 따뜻한 손편지에 마음이 뭉클해졌다.,기쁨,감동,설탕 스틱 껴준거 센스 백점 만점에 천점,20.0,NaN,기쁨,감동
1,1,비가 오는데도 친구가 내 좋아하는 카페까지 우산 들고 따라와줘서 마음이 따뜻해졌어.,기쁨,감동,아쓰 산차이 기분 안 좋은 거 알아채고 산차이가 가고 싶다던 토끼집 데려온 거 감동,79.0,NaN,기쁨,감동
2,2,"햇살 아래 반짝이는 아이의 눈동자가 마치 작은 보석처럼 빛났다. 그 순간, 세상 모...",기쁨,감동,신데렐라 드레스는 다시 봐도 너무 아름다워. 사람에게 꿈의 물결을 입히다니요.,104.0,NaN,기쁨,감동
3,3,이번 전시회 준비하면서 철저하게 세부까지 챙겨준 덕분에 모든 게 완벽하게 마무리돼서...,기쁨,감동,와 민희진 씨 애들 숙소 스타일링까지 맡기면서 신경써 준 거 진짜 좀 대단하네,107.0,NaN,기쁨,감동
4,4,비 오는 날 낯선 사람이 내게 담요를 건네며 추위 걱정해 줬다. 마음이 따뜻해져서 ...,기쁨,감동,개감동인 거 자기가 쓰고 있던 우산 나 주고\n자기가 비 맞아가면서 뒤집어준 거야\...,137.0,NaN,기쁨,감동


In [27]:
def embeddings_model():
  """
  임베딩 모델 초기화
  """
  model = SentenceTransformer("dragonkue/snowflake-arctic-embed-l-v2.0-ko") 
  vec_dim = len(model.encode("dummy_text"))
  print(f"모델 차원: {vec_dim}")
  return model

In [28]:
embeddings_model = embeddings_model()

모델 차원: 1024


In [51]:
# 기존 변수 초기화 (중립 데이터 제거로 인한 크기 불일치 방지)
vars_to_reset = ['X', 'y', 'y_encoded', 'X_combined', 'y_cat2', 'y_cat2_encoded', 'le', 'le_cat2', 'cat1_encoder']
for var_name in vars_to_reset:
    if var_name in locals():
        del locals()[var_name]
        print(f"변수 {var_name} 초기화됨")

print("✅ 기존 변수들이 초기화되었습니다.")

# 데이터 정보 확인
data.info()

변수 X 초기화됨
변수 y 초기화됨
✅ 기존 변수들이 초기화되었습니다.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3359 entries, 0 to 3358
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          3359 non-null   int64  
 1   generator_context   3359 non-null   object 
 2   category1           3359 non-null   object 
 3   category2           3359 non-null   object 
 4   input_context       3359 non-null   object 
 5   original_index      664 non-null    float64
 6   augmentation_index  2695 non-null   float64
 7   re_category1        3359 non-null   object 
 8   re_category2        3359 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 236.3+ KB


In [52]:
# 중립 데이터 제거 후 벡터 생성
print("📝 필터링된 데이터로 벡터 생성...")
data['vector'] = data['generator_context'].apply(lambda x: embeddings_model.encode(x).tolist())
print(f"✅ 벡터 생성 완료: {len(data)}개")

📝 필터링된 데이터로 벡터 생성...
✅ 벡터 생성 완료: 3359개


In [8]:
# 코사인 유사도를 사용한 Top3 유사한 샘플 찾기
print("🔍 코사인 유사도를 사용한 Top3 유사한 샘플 찾기")
print("="*60)

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 벡터 데이터를 numpy 배열로 변환
print("📊 벡터 데이터 준비 중...")
vectors = np.vstack(data['vector'].values)
print(f"벡터 행렬 크기: {vectors.shape}")

# 메모리 효율성을 위해 배치로 처리
batch_size = 100
n_samples = len(data)

print(f"🧮 코사인 유사도 계산 중... (총 {n_samples}개 샘플을 배치 단위로 처리)")

# 새로운 컬럼들을 위한 리스트 초기화
top1_context = []
top1_category = []
top2_context = []
top2_category = []
top3_context = []
top3_category = []

# 배치별로 처리
for batch_start in range(0, n_samples, batch_size):
    batch_end = min(batch_start + batch_size, n_samples)
    print(f"처리 중: {batch_start+1}-{batch_end} / {n_samples}")
    
    # 현재 배치에 대한 유사도 계산
    batch_vectors = vectors[batch_start:batch_end]
    similarities = cosine_similarity(batch_vectors, vectors)
    
    # 각 샘플에 대해 Top3 유사한 샘플 찾기
    for i in range(batch_end - batch_start):
        current_idx = batch_start + i
        sim_row = similarities[i]
        
        # 자기 자신과의 유사도를 -1로 설정 (제외)
        sim_row[current_idx] = -1
        
        # 유사도가 높은 순서로 정렬 (인덱스 반환)
        top_indices = np.argsort(sim_row)[::-1][:3]  # 상위 3개
        top_similarities = sim_row[top_indices]
        
        # Top 1
        idx1 = top_indices[0]
        sim1 = top_similarities[0]
        top1_context.append(data.iloc[idx1]['generator_context'])
        top1_category.append(f"{data.iloc[idx1]['re_category1']}_{data.iloc[idx1]['re_category2']}_{sim1:.4f}")
        
        # Top 2
        idx2 = top_indices[1]
        sim2 = top_similarities[1]
        top2_context.append(data.iloc[idx2]['generator_context'])
        top2_category.append(f"{data.iloc[idx2]['re_category1']}_{data.iloc[idx2]['re_category2']}_{sim2:.4f}")
        
        # Top 3
        idx3 = top_indices[2]
        sim3 = top_similarities[2]
        top3_context.append(data.iloc[idx3]['generator_context'])
        top3_category.append(f"{data.iloc[idx3]['re_category1']}_{data.iloc[idx3]['re_category2']}_{sim3:.4f}")

print("✅ Top3 유사한 샘플 찾기 완료")

# 새로운 컬럼들을 데이터프레임에 추가
print("📝 새로운 컬럼들을 데이터프레임에 추가 중...")
data['top1_context'] = top1_context
data['top1_category'] = top1_category
data['top2_context'] = top2_context
data['top2_category'] = top2_category
data['top3_context'] = top3_context
data['top3_category'] = top3_category

print("✅ 새로운 컬럼 추가 완료")

# 결과 확인
print(f"\n📋 업데이트된 데이터프레임 정보:")
print(f"데이터 크기: {data.shape}")
print(f"새로 추가된 컬럼: top1_context, top1_category, top2_context, top2_category, top3_context, top3_category")

# 샘플 확인
print(f"\n🔍 Top3 유사도 결과 샘플 (첫 3개):")
print("="*100)
for i in range(3):
    print(f"[샘플 {i+1}]")
    print(f"원본 텍스트: {data.iloc[i]['generator_context'][:80]}...")
    print(f"원본 카테고리: {data.iloc[i]['re_category1']}_{data.iloc[i]['re_category2']}")
    print()
    
    print(f"  Top1 유사 텍스트: {data.iloc[i]['top1_context'][:80]}...")
    print(f"  Top1 카테고리&유사도: {data.iloc[i]['top1_category']}")
    print()
    
    print(f"  Top2 유사 텍스트: {data.iloc[i]['top2_context'][:80]}...")
    print(f"  Top2 카테고리&유사도: {data.iloc[i]['top2_category']}")
    print()
    
    print(f"  Top3 유사 텍스트: {data.iloc[i]['top3_context'][:80]}...")
    print(f"  Top3 카테고리&유사도: {data.iloc[i]['top3_category']}")
    print("="*100)

# 유사도 분포 통계
print(f"\n📈 유사도 분포 통계:")
all_similarities = []
for i in range(len(data)):
    for j in [1, 2, 3]:
        category_col = f'top{j}_category'
        similarity_str = data.iloc[i][category_col].split('_')[-1]
        all_similarities.append(float(similarity_str))

similarities_array = np.array(all_similarities)
print(f"평균 유사도: {similarities_array.mean():.4f}")
print(f"최대 유사도: {similarities_array.max():.4f}")
print(f"최소 유사도: {similarities_array.min():.4f}")
print(f"표준편차: {similarities_array.std():.4f}")

# 카테고리 일치 분석
print(f"\n📊 Top3 유사 샘플들과 원본의 카테고리 일치 분석:")
category1_matches = [0, 0, 0]  # top1, top2, top3
category2_matches = [0, 0, 0]
both_matches = [0, 0, 0]

for i in range(len(data)):
    orig_cat1 = data.iloc[i]['re_category1']
    orig_cat2 = data.iloc[i]['re_category2']
    
    for j in range(3):
        top_category = data.iloc[i][f'top{j+1}_category']
        top_cat1, top_cat2, _ = top_category.split('_')
        
        if orig_cat1 == top_cat1:
            category1_matches[j] += 1
        if orig_cat2 == top_cat2:
            category2_matches[j] += 1
        if orig_cat1 == top_cat1 and orig_cat2 == top_cat2:
            both_matches[j] += 1

for j in range(3):
    print(f"Top{j+1} - Category1 일치: {category1_matches[j]}/{len(data)} ({category1_matches[j]/len(data)*100:.1f}%)")
    print(f"Top{j+1} - Category2 일치: {category2_matches[j]}/{len(data)} ({category2_matches[j]/len(data)*100:.1f}%)")
    print(f"Top{j+1} - 둘 다 일치: {both_matches[j]}/{len(data)} ({both_matches[j]/len(data)*100:.1f}%)")
    print()

print(f"🎉 코사인 유사도 기반 Top3 유사한 샘플 추가 완료!")
print(f"총 {len(data)}개 샘플에 대해 각각 Top3 유사한 샘플 정보가 추가되었습니다.")

# 컬럼 정보 최종 확인
print(f"\n📋 최종 데이터프레임 컬럼 목록:")
for col in data.columns:
    print(f"  - {col}")
    
print(f"\n데이터 저장 권장사항:")
print(f"data.to_excel('enhanced_data_with_similarity.xlsx', index=False)")
print(f"# 또는")
print(f"data.to_csv('enhanced_data_with_similarity.csv', index=False)")

🔍 코사인 유사도를 사용한 Top3 유사한 샘플 찾기
📊 벡터 데이터 준비 중...
벡터 행렬 크기: (3359, 1024)
🧮 코사인 유사도 계산 중... (총 3359개 샘플을 배치 단위로 처리)
처리 중: 1-100 / 3359
처리 중: 101-200 / 3359
처리 중: 201-300 / 3359
처리 중: 301-400 / 3359
처리 중: 401-500 / 3359
처리 중: 501-600 / 3359
처리 중: 601-700 / 3359
처리 중: 701-800 / 3359
처리 중: 801-900 / 3359
처리 중: 901-1000 / 3359
처리 중: 1001-1100 / 3359
처리 중: 1101-1200 / 3359
처리 중: 1201-1300 / 3359
처리 중: 1301-1400 / 3359
처리 중: 1401-1500 / 3359
처리 중: 1501-1600 / 3359
처리 중: 1601-1700 / 3359
처리 중: 1701-1800 / 3359
처리 중: 1801-1900 / 3359
처리 중: 1901-2000 / 3359
처리 중: 2001-2100 / 3359
처리 중: 2101-2200 / 3359
처리 중: 2201-2300 / 3359
처리 중: 2301-2400 / 3359
처리 중: 2401-2500 / 3359
처리 중: 2501-2600 / 3359
처리 중: 2601-2700 / 3359
처리 중: 2701-2800 / 3359
처리 중: 2801-2900 / 3359
처리 중: 2901-3000 / 3359
처리 중: 3001-3100 / 3359
처리 중: 3101-3200 / 3359
처리 중: 3201-3300 / 3359
처리 중: 3301-3359 / 3359
✅ Top3 유사한 샘플 찾기 완료
📝 새로운 컬럼들을 데이터프레임에 추가 중...
✅ 새로운 컬럼 추가 완료

📋 업데이트된 데이터프레임 정보:
데이터 크기: (3359, 16)
새로 추가된 컬럼: top1_context, t

In [ ]:
data.to_excel('enhanced_data_with_similarity.xlsx', index=False)

In [46]:
data = pd.read_excel(r'C:\Users\user\Desktop\SKN_AFTER_STUDY\data\enhanced_data_with_similarity.xlsx')

In [47]:
data[(data['re_category1']!=data['category1'])|(data['re_category2']!=data['category2'])].to_excel('dif.xlsx', index=False)

In [48]:
good_data = data[(data['re_category1']==data['category1'])&(data['re_category2']==data['category2'])]

In [13]:
re_label = pd.read_excel(r'C:\Users\user\Desktop\SKN_AFTER_STUDY\data\dif.xlsx')

In [49]:
data = pd.concat([good_data, re_label], axis=0, ignore_index=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3359 entries, 0 to 3358
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Unnamed: 0          3359 non-null   int64  
 1   generator_context   3359 non-null   object 
 2   category1           3359 non-null   object 
 3   category2           3359 non-null   object 
 4   input_context       3359 non-null   object 
 5   original_index      664 non-null    float64
 6   augmentation_index  2695 non-null   float64
 7   re_category1        3359 non-null   object 
 8   re_category2        3359 non-null   object 
 9   vector              3359 non-null   object 
 10  top1_context        3359 non-null   object 
 11  top1_category       3359 non-null   object 
 12  top2_context        3359 non-null   object 
 13  top2_category       3359 non-null   object 
 14  top3_context        3359 non-null   object 
 15  top3_category       3359 non-null   object 
dtypes: flo

In [50]:
data = data.iloc[:, :9]

### category1

In [54]:
# 필터링된 데이터로 벡터와 라벨 생성
print("데이터 상태 확인:")
print(f"필터링된 데이터 개수: {len(data)}")
print(f"벡터 타입: {type(data['vector'].iloc[0])}")
print(f"벡터 길이: {len(data['vector'].iloc[0])}")

# 벡터가 이미 리스트 형태라면 직접 numpy array로 변환
if isinstance(data['vector'].iloc[0], list):
    print("벡터가 리스트 형태입니다. 직접 변환합니다...")
    X = np.vstack(data['vector'].values)
    y = data['re_category1'].values  # 변경: category1 → re_category1
    print(f"X shape: {X.shape}")
    print(f"y shape: {y.shape}")
    
    # 크기 일치 확인
    if X.shape[0] == y.shape[0]:
        print("✅ X와 y의 크기가 일치합니다!")
    else:
        print(f"❌ 크기 불일치: X {X.shape[0]} vs y {y.shape[0]}")
    
    print("✅ 성공적으로 변환되었습니다!")
else:
    print("벡터 형태에 문제가 있습니다.")

데이터 상태 확인:
필터링된 데이터 개수: 3359
벡터 타입: <class 'list'>
벡터 길이: 1024
벡터가 리스트 형태입니다. 직접 변환합니다...
X shape: (3359, 1024)
y shape: (3359,)
✅ X와 y의 크기가 일치합니다!
✅ 성공적으로 변환되었습니다!


In [55]:
# 9. 실제 test_data로 모델 성능 평가

print("📁 실제 test_data 로드 및 평가\n")

# test_data 로드
test_data = pd.read_excel(r'C:\Users\user\Desktop\SKN_AFTER_STUDY\data\증강할데이터33.xlsx')
print("테스트 데이터 기본 정보:")
print(f"데이터 크기: {test_data.shape}")
print(f"컬럼들: {list(test_data.columns)}")
print("\n데이터 샘플:")
print(test_data.head())

# test_data에서 텍스트와 category1 컬럼 확인
print(f"\ntest_data 컬럼 확인:")
for col in test_data.columns:
    print(f"- {col}: {test_data[col].dtype}")

# 텍스트 컬럼과 category1 컬럼 식별 (컬럼명에 따라 조정 필요)
text_column = None
category1_column = None

# 가능한 텍스트 컬럼명들
possible_text_columns = ['context', 'text', 'content', 'sentence', '내용', '문장']
for col in test_data.columns:
    if any(keyword in col.lower() for keyword in possible_text_columns):
        text_column = col
        break

# 가능한 category1 컬럼명들
possible_cat1_columns = ['category1', 'cat1', 'label', '감정', '카테고리1']
for col in test_data.columns:
    if any(keyword in col.lower() for keyword in possible_cat1_columns):
        category1_column = col
        break

print(f"\n식별된 컬럼:")
print(f"텍스트 컬럼: {text_column}")
print(f"Category1 컬럼: {category1_column}")

if text_column and category1_column:
    print(f"\n✅ 필요한 컬럼들을 찾았습니다!")
    print(f"테스트 데이터 개수: {len(test_data)}")
    print(f"Category1 클래스들: {test_data[category1_column].unique()}")
else:
    print(f"\n❌ 필요한 컬럼을 찾을 수 없습니다. 수동으로 지정해주세요.")
    print("사용 가능한 컬럼들:")
    for i, col in enumerate(test_data.columns):
        print(f"{i}: {col}")

📁 실제 test_data 로드 및 평가

테스트 데이터 기본 정보:
데이터 크기: (664, 5)
컬럼들: ['index', 'context', 'annotations_split', 'category1', 'category2']

데이터 샘플:
   index  \
0      0   
1      1   
2      2   
3      3   
4      4   

                                                                                              context  \
0                                            보는동안 너무 행복했고 초콜렛이 너무 먹고싶었고 티모시가 잘생겼고 울어!!하는부분이 있어서 울었다네요   
1                                            어릴 때 가 보고 빕스는 거의 처음인데(기억에 없음) 지금 딸기축제 기간이라 만족스러운 식사 하고 옴   
2  미리 계좌로 환전해둔 돈을 해외에서 환전수수료 없이 인출 가능한 트레블로그라는 카드인데, 선택할 수 있는 디자인 중에 이 여권 스타일이 너무 센스 있고 유니크해서 마음에 든다.   
3                                                   요즘 번아웃도 자꾸 올라오고 무기력해서 종강하고 교류하기도 버거운 상태가 와부렀으요ㅠㅠ    
4                                    크라임씬 장똥민이 범행 도구 찾으려고 화장실 탱크 뒤지는데 거기에 진짜 똥 넣어놓은 거 진짜 웃겨 뒤지겠음ㅋㅋㅋㅋㅋ   

                                                                  annotations_split  \
0         [['기쁨', '만족감'], ['기쁨', '만족감'], ['기쁨', '감동'], ['기쁨', '감동'], ['

In [56]:
# 테스트 데이터를 위한 벡터 생성 및 라벨 준비
print(f"\n🔧 테스트 데이터 벡터 생성 및 라벨 준비...")

# 테스트 데이터에서 벡터 생성
test_texts = test_data['context'].values
print(f"테스트 텍스트 샘플:")
for i in range(3):
    print(f"{i+1}. {test_texts[i][:100]}...")

print(f"\n📊 테스트 데이터 벡터화 중...")
test_vectors = [embeddings_model.encode(text).tolist() for text in test_texts]
test_X = np.array(test_vectors)

# 테스트 라벨 준비
test_y_actual = test_data['category1'].values
test_y_actual_cat2 = test_data['category2'].values

print(f"✅ 테스트 데이터 준비 완료:")
print(f"test_X shape: {test_X.shape}")
print(f"test_y_actual shape: {test_y_actual.shape}")
print(f"test_y_actual_cat2 shape: {test_y_actual_cat2.shape}")
print(f"테스트 데이터 category1 클래스: {len(np.unique(test_y_actual))}개")
print(f"테스트 데이터 category2 클래스: {len(np.unique(test_y_actual_cat2))}개")


🔧 테스트 데이터 벡터 생성 및 라벨 준비...
테스트 텍스트 샘플:
1. 보는동안 너무 행복했고 초콜렛이 너무 먹고싶었고 티모시가 잘생겼고 울어!!하는부분이 있어서 울었다네요...
2. 어릴 때 가 보고 빕스는 거의 처음인데(기억에 없음) 지금 딸기축제 기간이라 만족스러운 식사 하고 옴...
3. 미리 계좌로 환전해둔 돈을 해외에서 환전수수료 없이 인출 가능한 트레블로그라는 카드인데, 선택할 수 있는 디자인 중에 이 여권 스타일이 너무 센스 있고 유니크해서 마음에 든다....

📊 테스트 데이터 벡터화 중...
✅ 테스트 데이터 준비 완료:
test_X shape: (664, 1024)
test_y_actual shape: (664,)
test_y_actual_cat2 shape: (664,)
테스트 데이터 category1 클래스: 10개
테스트 데이터 category2 클래스: 64개


In [57]:
# 복합 라벨 방식 AutoGluon 모델 훈련
print(f"\n🤖 복합 라벨 방식 AutoGluon 모델 훈련 시작...")

# 필요한 라이브러리 임포트
try:
    from autogluon.tabular import TabularPredictor
    print("✅ AutoGluon 라이브러리 로드 성공")
    autogluon_available = True
except ImportError:
    print("❌ AutoGluon이 설치되지 않았습니다. 다음 명령으로 설치하세요:")
    print("pip install autogluon")
    autogluon_available = False
    raise ImportError("AutoGluon이 필요합니다")

import os
import warnings
warnings.filterwarnings('ignore')

# 복합 라벨 생성
combined_labels = y + "_" + data['re_category2'].values
print(f"복합 라벨 예시: {combined_labels[:5]}")
print(f"총 복합 라벨 종류: {len(np.unique(combined_labels))}개")

# 복합 라벨 분포 확인
label_counts = pd.Series(combined_labels).value_counts()
print(f"\n📈 복합 라벨 상위 10개 분포:")
print(label_counts.head(10))

# 희소한 라벨 확인 (3개 미만)
rare_labels = label_counts[label_counts < 3]
if len(rare_labels) > 0:
    print(f"\n⚠️ 희소한 라벨 ({len(rare_labels)}개): {list(rare_labels.index[:10])}")
    print("이러한 라벨들은 훈련이 어려울 수 있습니다.")

# AutoGluon용 훈련 데이터 준비
print(f"\n📊 AutoGluon 훈련 데이터 준비...")
train_combined_df = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
train_combined_df['combined_label'] = combined_labels

print(f"훈련 데이터 크기: {train_combined_df.shape}")

# AutoGluon 모델 저장 경로
combined_save_path = "autogluon_combined_f1_model"
if os.path.exists(combined_save_path):
    import shutil
    shutil.rmtree(combined_save_path)
    print(f"기존 모델 폴더 {combined_save_path} 삭제됨")

# AutoGluon 복합 라벨 모델 훈련 (F1 스코어 최적화)
print(f"\n🚀 AutoGluon F1-Score 최적화 모델 훈련 시작...")
print("="*60)

combined_predictor = TabularPredictor(
    label='combined_label',
    path=combined_save_path,
    eval_metric='f1_macro',  # F1-macro 점수로 최적화
    problem_type='multiclass'
)

print("복합 라벨 모델들 훈련 중... (예상 소요 시간: 5-10분)")
combined_predictor = combined_predictor.fit(
    train_data=train_combined_df,
    time_limit=600,  # 10분 제한 (더 좋은 성능을 위해)
    presets='best_quality',  # 최고 품질 프리셋
    num_bag_folds=5,  # 5-fold 교차 검증
    num_bag_sets=1,
    num_stack_levels=2,  # 스태킹 레벨 증가
    hyperparameters='default'  # 다중 클래스 최적화
)

print("✅ AutoGluon 복합 라벨 모델 훈련 완료!")

# 모델 리더보드 출력
print(f"\n📋 AutoGluon 모델 리더보드 (F1-macro 기준):")
leaderboard = combined_predictor.leaderboard()
print(leaderboard.head(10))

# 최고 성능 모델 정보
best_model = leaderboard.iloc[0]['model']
best_f1_score = leaderboard.iloc[0]['score_val']
print(f"\n🏆 최고 성능 모델: {best_model}")
print(f"검증 F1-macro Score: {best_f1_score:.4f}")

# 테스트 데이터 예측
print(f"\n🎯 AutoGluon 모델로 테스트 데이터 예측...")
test_combined_df = pd.DataFrame(test_X, columns=[f'feature_{i}' for i in range(test_X.shape[1])])

# 예측 수행
test_pred_combined_labels = combined_predictor.predict(test_combined_df)
test_pred_proba = combined_predictor.predict_proba(test_combined_df)

print(f"✅ AutoGluon 모델 예측 완료!")

# 예측된 복합 라벨을 개별 카테고리로 분리
test_pred_split = test_pred_combined_labels.str.split('_', expand=True)
test_pred_cat1 = test_pred_split[0].values
test_pred_cat2 = test_pred_split[1].values

# 실제 라벨도 복합 라벨 형태로 생성 (비교용)
test_actual_combined_labels = test_y_actual + "_" + test_y_actual_cat2

# 성능 평가
from sklearn.metrics import accuracy_score, f1_score, classification_report
cat1_accuracy = accuracy_score(test_y_actual, test_pred_cat1)
cat2_accuracy = accuracy_score(test_y_actual_cat2, test_pred_cat2)
combined_accuracy = accuracy_score(test_actual_combined_labels, test_pred_combined_labels)

# F1 스코어 계산
f1_cat1 = f1_score(test_y_actual, test_pred_cat1, average='macro')
f1_cat2 = f1_score(test_y_actual_cat2, test_pred_cat2, average='macro') 
f1_combined = f1_score(test_actual_combined_labels, test_pred_combined_labels, average='macro')

print(f"\n📊 AutoGluon 복합 라벨 모델 성능:")
print("="*50)
print(f"정확도:")
print(f"  Category1: {cat1_accuracy:.4f} ({cat1_accuracy*100:.2f}%)")
print(f"  Category2: {cat2_accuracy:.4f} ({cat2_accuracy*100:.2f}%)")
print(f"  복합 라벨: {combined_accuracy:.4f} ({combined_accuracy*100:.2f}%)")

print(f"\nF1-Score (macro):")
print(f"  Category1: {f1_cat1:.4f}")
print(f"  Category2: {f1_cat2:.4f}")
print(f"  복합 라벨: {f1_combined:.4f}")

# 두 카테고리 모두 정답인 경우
both_correct = (test_pred_cat1 == test_y_actual) & (test_pred_cat2 == test_y_actual_cat2)
both_accuracy = both_correct.mean()
print(f"\n🎯 두 카테고리 모두 정답: {both_accuracy:.4f} ({both_accuracy*100:.2f}%)")
print(f"정답 개수: {both_correct.sum()}/{len(test_y_actual)}")

print(f"\n✅ AutoGluon F1-최적화 모델 성능 평가 완료!")

# 모델 저장 정보
print(f"\n💾 모델 저장 위치: {combined_save_path}")
print(f"📥 모델 재로드 방법:")
print(f"combined_predictor = TabularPredictor.load('{combined_save_path}')")

# 최종 성과 요약
print(f"\n🎉 최종 성과 요약:")
print(f"🏆 최고 성능 모델: {best_model}")
print(f"📈 검증 F1-macro: {best_f1_score:.4f}")
print(f"📊 테스트 F1-macro: {f1_combined:.4f}")
print(f"🎯 두 카테고리 동시 정답률: {both_accuracy*100:.2f}%")


🤖 복합 라벨 방식 AutoGluon 모델 훈련 시작...
✅ AutoGluon 라이브러리 로드 성공
복합 라벨 예시: ['기쁨_감동' '기쁨_감동' '기쁨_감동' '기쁨_감동' '기쁨_감동']
총 복합 라벨 종류: 76개

📈 복합 라벨 상위 10개 분포:
슬픔_무기력      98
수치심_부끄러움    96
슬픔_외로움      90
기쁨_기대감      84
두려움_놀람      83
기쁨_공감       83
기쁨_편안함      80
슬픔_허망       78
기쁨_안정감      73
분노_불쾌       72
Name: count, dtype: int64

⚠️ 희소한 라벨 (7개): ['중립_공감', '수치심_수치심', '미움(상대방)_불쾌', '슬픔_답답함', '슬픔_죄책감', '사랑_공감', '슬픔_공감']
이러한 라벨들은 훈련이 어려울 수 있습니다.

📊 AutoGluon 훈련 데이터 준비...
훈련 데이터 크기: (3359, 1025)


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.11
Operating System:   Windows
Platform Machine:   AMD64
Platform Version:   10.0.19045
CPU Count:          16
Memory Avail:       14.78 GB / 31.91 GB (46.3%)
Disk Space Avail:   73.07 GB / 465.12 GB (15.7%)
Presets specified: ['best_quality']
Using hyperparameters preset: hyperparameters='default'
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=2, num_bag_folds=5, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is use

기존 모델 폴더 autogluon_combined_f1_model 삭제됨

🚀 AutoGluon F1-Score 최적화 모델 훈련 시작...
복합 라벨 모델들 훈련 중... (예상 소요 시간: 5-10분)


Leaderboard on holdout data (DyStack):
                    model  score_holdout  score_val eval_metric  pred_time_test  pred_time_val    fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0     WeightedEnsemble_L2       0.611152   0.626684    f1_macro        1.280977       0.348022   54.318815                 0.004001                0.003001           0.272061            2       True          4
1  NeuralNetFastAI_BAG_L1       0.608330   0.622798    f1_macro        1.072931       0.185041   14.538218                 1.072931                0.185041          14.538218            1       True          1
2     WeightedEnsemble_L4       0.605447   0.637328    f1_macro        1.531044       0.692788   95.697352                 0.007002                0.004000           0.352452            4       True         10
3  NeuralNetFastAI_BAG_L3       0.587420   0.610210    f1_macro        1.692081       0.909314  109.469433               

✅ AutoGluon 복합 라벨 모델 훈련 완료!

📋 AutoGluon 모델 리더보드 (F1-macro 기준):
                     model  score_val eval_metric  pred_time_val    fit_time  \
0      WeightedEnsemble_L2   0.621278    f1_macro       1.250298  352.629694   
1   NeuralNetFastAI_BAG_L1   0.611499    f1_macro       0.189046   14.384632   
2        LightGBMXT_BAG_L1   0.538246    f1_macro       0.739181  334.311806   
3  RandomForestGini_BAG_L1   0.371932    f1_macro       0.319072    3.574869   
4          LightGBM_BAG_L1   0.345299    f1_macro       0.142033   63.536590   

   pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  \
0                0.003000           0.358388            2       True   
1                0.189046          14.384632            1       True   
2                0.739181         334.311806            1       True   
3                0.319072           3.574869            1       True   
4                0.142033          63.536590            1       True   

   fit_order  
0      

In [ ]:
# AutoGluon 복합 라벨 모델 데모 텍스트 예측 시스템
print("🎯 AutoGluon 복합 라벨 모델 데모 텍스트 예측 시스템")
print("="*60)

def predict_emotion_autogluon(text, predictor=combined_predictor, embedder=embeddings_model):
    """
    AutoGluon 복합 라벨 모델을 사용하여 입력 텍스트의 감정을 예측하는 함수
    
    Args:
        text (str): 예측할 텍스트
        predictor: AutoGluon 예측기
        embedder: 텍스트 임베딩 모델
    
    Returns:
        dict: 예측 결과
    """
    # 텍스트를 벡터로 변환
    text_vector = embedder.encode(text)
    
    # AutoGluon 입력 형태로 변환
    text_df = pd.DataFrame([text_vector], columns=[f'feature_{i}' for i in range(len(text_vector))])
    
    # 복합 라벨 예측
    combined_pred = predictor.predict(text_df).iloc[0]
    
    # 예측 확률
    pred_proba = predictor.predict_proba(text_df)
    combined_confidence = pred_proba.iloc[0].max()
    
    # 복합 라벨을 개별 카테고리로 분리
    cat1_pred, cat2_pred = combined_pred.split('_')
    
    # 상위 5개 복합 라벨 후보
    top5_proba = pred_proba.iloc[0].nlargest(5)
    top5_predictions = [(label, score) for label, score in top5_proba.items()]
    
    return {
        'text': text,
        'combined_label': combined_pred,
        'category1': cat1_pred,
        'category2': cat2_pred,
        'confidence': combined_confidence,
        'top5_predictions': top5_predictions
    }

def print_autogluon_prediction(result):
    """AutoGluon 복합 라벨 예측 결과를 보기 좋게 출력하는 함수"""
    print(f"📝 입력 텍스트: {result['text']}")
    print(f"🎯 AutoGluon 예측 결과:")
    print(f"   복합 라벨: {result['combined_label']} (신뢰도: {result['confidence']:.3f})")
    print(f"   Category1 (대분류): {result['category1']}")
    print(f"   Category2 (세분류): {result['category2']}")
    
    print(f"📊 상위 5개 복합 라벨 후보:")
    for i, (label, score) in enumerate(result['top5_predictions']):
        cat1, cat2 = label.split('_')
        print(f"   {i+1}. {label} ({cat1}|{cat2}): {score:.3f}")
    print()

# 다양한 감정 표현 데모 텍스트들
demo_texts = [
    # "오늘 친구가 생일선물을 깜짝 준비해줘서 정말 감동받았어요. 너무 고마워서 눈물이 났어요.",
    # "시험에서 떨어져서 너무 실망스럽고 우울해요. 앞으로 어떻게 해야 할지 모르겠어요.",
    # "새로운 직장에서 첫날인데 너무 떨려요. 잘할 수 있을까 걱정이 많이 돼요.",
    # "연인과 헤어져서 너무 화나고 원망스러워요. 배신감이 들어서 잠도 못 자겠어요.",
    # "로또에 당첨되어서 너무 기뻐요! 꿈만 같아서 믿기지가 않아요.",
    # "새로운 취미를 시작하게 되어서 설레고 기대돼요. 뭔가 새로운 도전이라 흥미로워요.",
    # "혼자 집에 있는데 갑자기 이상한 소리가 들려서 무서워요. 심장이 빨리 뛰어요.",
    # "부모님께서 제 꿈을 응원해주셔서 너무 든든하고 사랑받는 기분이에요.",
    "손수 만든 작은 포장을 끝냈다. 계획대로 마무리돼 마음속으로 은근히 기뻤다.",
    "작은 모형 배를 조립해 완성했다. 계획대로 맞아 마음이 은은히 기뻤다.",
    "인파가 북적이는 지하철역 입구에서 큰 소리로 침을 뱉으며 지나가는 사람들이 짜증이 났다.",
    "정말 믿음이 깨져버렸어. 아무리 약속해도 뒤통수를 맞는 기분, 이젠 누굴 믿어야 할지 모르겠어.",
    "저녁 모임에서 내 기여를 무시한 채 다른 사람이 칭찬받는 걸 보며 마음속에 차오르는 싸늘한 불쾌감이 멈추지 않았다.",
    "식당에서 그가 주문을 받고도 무표정하게 돌아서자, 나는 깊은 쓸쓸함과 함께 차가운 벽에 막힌 기분이 들었다."
]

print(f"🌟 AutoGluon 복합 라벨 모델을 사용한 감정 텍스트 예측 데모:")
print(f"총 {len(demo_texts)}개의 예시 텍스트로 감정 예측을 수행합니다.")
print(f"🏆 사용 모델: {best_model} (F1-macro: {best_f1_score:.4f})")
print("="*80)

# 각 데모 텍스트에 대해 AutoGluon 복합 라벨 예측 수행
for i, demo_text in enumerate(demo_texts, 1):
    print(f"[예시 {i}]")
    result = predict_emotion_autogluon(demo_text)
    print_autogluon_prediction(result)

🎯 AutoGluon 복합 라벨 모델 데모 텍스트 예측 시스템
🌟 AutoGluon 복합 라벨 모델을 사용한 감정 텍스트 예측 데모:
총 7개의 예시 텍스트로 감정 예측을 수행합니다.
🏆 사용 모델: WeightedEnsemble_L2 (F1-macro: 0.6213)
[예시 1]
📝 입력 텍스트: 손수 만든 작은 포장을 끝냈다. 계획대로 마무리돼 마음속으로 은근히 기뻤다.
🎯 AutoGluon 예측 결과:
   복합 라벨: 기쁨_만족감 (신뢰도: 0.609)
   Category1 (대분류): 기쁨
   Category2 (세분류): 만족감
📊 상위 5개 복합 라벨 후보:
   1. 기쁨_만족감 (기쁨|만족감): 0.609
   2. 기쁨_기대감 (기쁨|기대감): 0.084
   3. 기쁨_자랑스러움 (기쁨|자랑스러움): 0.033
   4. 기쁨_감동 (기쁨|감동): 0.018
   5. 슬픔_외로움 (슬픔|외로움): 0.014

[예시 2]
📝 입력 텍스트: 학습 노트를 마치고 정리했다. 차곡차곡 쌓인 흔적이 마음속으로 만족스러웠다.
🎯 AutoGluon 예측 결과:
   복합 라벨: 기쁨_만족감 (신뢰도: 0.541)
   Category1 (대분류): 기쁨
   Category2 (세분류): 만족감
📊 상위 5개 복합 라벨 후보:
   1. 기쁨_만족감 (기쁨|만족감): 0.541
   2. 기쁨_자랑스러움 (기쁨|자랑스러움): 0.049
   3. 기쁨_자신감 (기쁨|자신감): 0.028
   4. 기쁨_고마움 (기쁨|고마움): 0.028
   5. 기쁨_기대감 (기쁨|기대감): 0.023

[예시 3]
📝 입력 텍스트: 작은 모형 배를 조립해 완성했다. 계획대로 맞아 마음이 은은히 기뻤다.
🎯 AutoGluon 예측 결과:
   복합 라벨: 기쁨_만족감 (신뢰도: 0.641)
   Category1 (대분류): 기쁨
   Category2 (세분류): 만족감
📊 상위 5개 복합 라벨 후보:
   1. 기쁨_만족감 (기쁨|만족감): 0.641
   

In [59]:
# 사용자 입력 텍스트 예측 (AutoGluon 복합 라벨 모델)
print("🔍 AutoGluon 복합 라벨 모델을 사용한 사용자 입력 텍스트 감정 예측")
print("="*60)
print("아래 코드를 수정해서 원하는 텍스트의 감정을 예측해보세요!")
print()

# 여기에 원하는 텍스트를 입력하세요
custom_text = "친구들과 함께 여행을 가서 정말 즐거웠어요. 오랜만에 스트레스가 풀렸어요."

print(f"🎨 사용자 입력 텍스트 예측:")
print("-"*50)
custom_result = predict_emotion_autogluon(custom_text)
print_autogluon_prediction(custom_result)

print("💡 다른 텍스트로 테스트하려면:")
print("1. 위의 'custom_text' 변수에 원하는 텍스트를 입력하세요")
print("2. 셀을 다시 실행하세요")
print()
print("📌 예측 함수 사용법:")
print("result = predict_emotion_autogluon('여기에 텍스트 입력')")
print("print_autogluon_prediction(result)")
print()

# AutoGluon 복합 라벨 모델 정보 요약
print("📋 AutoGluon 복합 라벨 모델 정보 요약:")
print("="*60)
print(f"📊 훈련 데이터: {len(data)}개 샘플")
print(f"🏷️ 복합 라벨 클래스: {len(combined_predictor.class_labels)}개")

# 복합 라벨 클래스 예시 표시
sample_labels = list(combined_predictor.class_labels)[:10]
print(f"   - 예시: {', '.join(sample_labels)}{'...' if len(combined_predictor.class_labels) > 10 else ''}")

print(f"🤖 임베딩 모델: {embeddings_model.get_sentence_embedding_dimension()}차원")
print(f"🏆 최고 성능 모델: {best_model}")

print(f"\n🎯 테스트 성능:")
print(f"   - Category1 정확도: {cat1_accuracy*100:.2f}%")
print(f"   - Category2 정확도: {cat2_accuracy*100:.2f}%") 
print(f"   - 복합 라벨 정확도: {combined_accuracy*100:.2f}%")
print(f"   - 두 카테고리 모두 정답: {both_accuracy*100:.2f}%")

print(f"\n📈 F1-Score (macro):")
print(f"   - Category1: {f1_cat1:.4f}")
print(f"   - Category2: {f1_cat2:.4f}")
print(f"   - 복합 라벨: {f1_combined:.4f}")
print(f"   - 검증 F1-macro: {best_f1_score:.4f}")

print(f"\n🎯 AutoGluon 복합 라벨 모델의 장점:")
print("✅ F1-macro 점수 최적화로 불균형 클래스 성능 향상")
print("✅ 자동 하이퍼파라미터 튜닝 및 앙상블")
print("✅ 여러 알고리즘 조합으로 최고 성능 달성")
print("✅ 카테고리 간 의존성을 직접 학습")
print("✅ 5-fold 교차 검증으로 일반화 성능 보장")
print("✅ 스태킹 앙상블로 예측 안정성 향상")

print(f"\n💾 모델 재사용:")
print(f"모델 저장 위치: {combined_save_path}")
print("loaded_predictor = TabularPredictor.load('autogluon_combined_f1_model')")
print("prediction = loaded_predictor.predict(new_data)")

🔍 AutoGluon 복합 라벨 모델을 사용한 사용자 입력 텍스트 감정 예측
아래 코드를 수정해서 원하는 텍스트의 감정을 예측해보세요!

🎨 사용자 입력 텍스트 예측:
--------------------------------------------------
📝 입력 텍스트: 친구들과 함께 여행을 가서 정말 즐거웠어요. 오랜만에 스트레스가 풀렸어요.
🎯 AutoGluon 예측 결과:
   복합 라벨: 기쁨_즐거움 (신뢰도: 0.331)
   Category1 (대분류): 기쁨
   Category2 (세분류): 즐거움
📊 상위 5개 복합 라벨 후보:
   1. 기쁨_즐거움 (기쁨|즐거움): 0.331
   2. 기쁨_만족감 (기쁨|만족감): 0.131
   3. 기쁨_편안함 (기쁨|편안함): 0.113
   4. 기쁨_신명남 (기쁨|신명남): 0.038
   5. 기쁨_공감 (기쁨|공감): 0.032

💡 다른 텍스트로 테스트하려면:
1. 위의 'custom_text' 변수에 원하는 텍스트를 입력하세요
2. 셀을 다시 실행하세요

📌 예측 함수 사용법:
result = predict_emotion_autogluon('여기에 텍스트 입력')
print_autogluon_prediction(result)

📋 AutoGluon 복합 라벨 모델 정보 요약:
📊 훈련 데이터: 3359개 샘플
🏷️ 복합 라벨 클래스: 76개
   - 예시: 기쁨_감동, 기쁨_고마움, 기쁨_공감, 기쁨_기대감, 기쁨_놀람, 기쁨_만족감, 기쁨_반가움, 기쁨_신뢰감, 기쁨_신명남, 기쁨_안정감...
🤖 임베딩 모델: 1024차원
🏆 최고 성능 모델: WeightedEnsemble_L2

🎯 테스트 성능:
   - Category1 정확도: 53.31%
   - Category2 정확도: 34.34%
   - 복합 라벨 정확도: 32.08%
   - 두 카테고리 모두 정답: 32.08%

📈 F1-Score (macro):
   - Category1: 0.3936
   - Category2